# setup



In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle
!kaggle competitions download -c 11785-spring2021-hw2p2s1-face-classification
!pwd

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
from PIL import Image
import PIL
import random
import tensorflow as tf
from torchvision.transforms import Compose, RandomCrop, ToTensor, ToPILImage, CenterCrop, Resize
import matplotlib.pyplot as plt
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:

normalize = torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
transforms = torchvision.transforms.Compose([

    torchvision.transforms.ColorJitter(hue=.05, saturation=.05),
    torchvision.transforms.RandomHorizontalFlip(),
    #torchvision.transforms.RandomRotation(20, resample=PIL.Image.BILINEAR),
    torchvision.transforms.ToTensor(),
    normalize
])
train_dataset = torchvision.datasets.ImageFolder(root='train_data', 
                                                 transform = transforms)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, 
                                               shuffle=True, num_workers=1)

In [ ]:
transforms = torchvision.transforms.Compose([
                                             torchvision.transforms.ToTensor(),
                                             normalize
])

val_dataset = torchvision.datasets.ImageFolder(root='val_data', 
                                                 transform= transforms)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=128, 
                                               shuffle=False, num_workers=1)

# training


In [2]:

class SELayer(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

In [3]:
class SimpleResidualBlock(torch.nn.Module):
    def __init__(self, in_channels, out_channels ,stride=1, downsample = False):
        super(SimpleResidualBlock, self).__init__()
        self.downsample = downsample
        self.expansion = 1
        self.conv1 = nn.Conv2d(in_channels, out_channels,kernel_size = 1,stride = 1, padding = 0, bias = False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(out_channels,out_channels,kernel_size = 3,stride = stride, padding = 1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.se = SELayer(out_channels, out_channels)
        
        self.shortcut = nn.Sequential()
        if downsample == False:
            self.shortcut = nn.Identity()
        else:
     
            self.shortcut = nn.Conv2d(in_channels,out_channels, kernel_size=1, stride=stride, bias = False)
            self.bn3 = nn.BatchNorm2d(out_channels)
       # self.relu = nn.ReLU(inplace = True)
    
    def forward(self, x):
      
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.se(out)

        if self.downsample == True:
          shortcut = self.bn3(self.shortcut(x))
        else:
          shortcut = self.shortcut(x)
       
        out = self.relu(out + shortcut)
        #out = out + shortcut
        
        return out

In [4]:
class Bottleneck(torch.nn.Module):
    def __init__(self, in_channels, out_channels ,stride=1, downsample = False):
        super(Bottleneck, self).__init__()
        self.downsample = downsample
        self.expansion = 4
        self.conv1 = nn.Conv2d(in_channels, out_channels,kernel_size = 1,stride = 1, padding = 0, bias = False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(out_channels,out_channels,kernel_size = 3,stride = stride, padding = 1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels*self.expansion, kernel_size = 1, stride = 1, bias = False)
        self.bn3 = nn.BatchNorm2d(out_channels*self.expansion)
        
        self.shortcut = nn.Sequential()
        if downsample == False:
            self.shortcut = nn.Identity()
        else:
  
            self.shortcut = nn.Conv2d(out_channels,out_channels*self.expansion, kernel_size=1, stride=stride, bias = False)
            self.bn4 = nn.BatchNorm2d(out_channels*self.expansion)
       # self.relu = nn.ReLU(inplace = True)
    
    def forward(self, x):
      
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample == True:
          shortcut = self.bn4(self.shortcut(x))
        else:
          shortcut = self.shortcut(x)
       
        out = self.relu(out + shortcut)
        #out = out + shortcut
        
        return out

In [5]:

class ResNet(torch.nn.Module):
  def __init__(self, block, layers,num_classes, feat_dim):
    super(ResNet, self).__init__()
    self.in_channels = 32
    self.conv1 = nn.Conv2d(3,32, kernel_size = 3, stride = 1, padding = 1, bias = False )
    self.bn1 = nn.BatchNorm2d(32)
    
    self.relu = nn.ReLU()
    #self.maxpool = nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 1)

 
    #for resnet layers
    self.layer1 = self.sublayer(block, layers[0], channel_out = 32, stride = 1)
    self.layer2 = self.sublayer(block, layers[1], channel_out = 48, stride = 1)
    self.layer3 = self.sublayer(block, layers[2], channel_out = 96, stride = 1)
    self.layer4 = self.sublayer(block, layers[3], channel_out = 192, stride = 2) 
    self.bn2 = nn.BatchNorm2d(192*block(2000,4000).expansion)
   
    self.avgpool =  nn.AdaptiveAvgPool2d((1, 1))
    self.flatten = nn.Flatten()
    self.dropout = nn.Dropout(p= 0.4)
    self.linear = nn.Linear(self.in_channels, feat_dim)
    
    self.linear_output = nn.Linear(192*block(2000, 4000).expansion, num_classes)
    
    
  
  def sublayer(self, block , num_block, channel_out, stride ):
   
    expansion = block(64,64,1).expansion
    layers = []
    for s in range(num_block):
      if s == 0:
        if expansion*channel_out != self.in_channels: # test if the in channel and out channel are the same
          layers.append(block(self.in_channels, channel_out, downsample = True))
        else:
          layers.append(block(channel_out, channel_out))
        self.in_channels = channel_out
      else:
        layers.append(block(self.in_channels*expansion, channel_out, stride, downsample = True))
    self.in_channels = channel_out * expansion
    return nn.Sequential(*layers)

  def forward(self, x, return_embedding = False):

    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    #x = self.maxpool(x)

    x = self.layer1(x)  
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)
    x = self.bn2(x)
    x = self.avgpool(x)
   
    x = self.flatten(x)
    x = self.dropout(x)
    
    embedding_out = self.relu(self.linear(x))

    output = self.linear_output(x)
    if return_embedding:
      return embedding_out, output
    else:
      return output





In [6]:


def ResNet34(num_classes, feat_dim):
  return ResNet(SimpleResidualBlock, [3,4,6,3],  num_classes, feat_dim)

def ResNet50(num_classes, feat_dim):
  return ResNet(Bottleneck, [3,4,6,3],  num_classes, feat_dim)

def ResNet50(num_classes, feat_dim):
  return ResNet(Bottleneck, [3,8,36,3],  num_classes, feat_dim)

In [ ]:
def training(epoch, network):
  cropsize_list = [24, 36, 48, 54, 64]
  standard = 0.3
  

  for epoch in range(numEpochs):
      
      network.train()
      avg_loss = 0.0
      num_correct = 0

      for batch_num, (x, y) in enumerate(train_dataloader):
          optimizer.zero_grad()
          
          x, y = x.to(device), y.to(device)

          #crop the image
          cropsize = random.choice(cropsize_list) #randomly center crop
          center_crop = torchvision.transforms.CenterCrop(cropsize)
          crop_image = center_crop(x)
          
          #put images to network
          outputs = network(crop_image)

          loss = criterion(outputs, y.long())
          loss.backward()
          optimizer.step()
          
          avg_loss += loss.item()
          if batch_num % 99 == 1:
              print('loss', avg_loss/100)
              avg_loss = 0.0
       
          #print(torch.argmax(outputs, axis = 1)[0],y[0])
          num_correct += (torch.argmax(outputs, axis=1) == y).sum().item()
          del x
          del y
          del crop_image
          del outputs
          torch.cuda.empty_cache()
      
      
      print('Epoch: {}, Training Accuracy: {:.2f}'.format(epoch, num_correct / len(train_dataset)))
      print('train loss',avg_loss)
      
      # Validate
      avg_loss = 0
      num_correct = 0
      network.eval()
      with torch.no_grad():
        cropsize_list = [24, 36, 48, 54, 64]

        for batch_num, (x, y) in enumerate(val_dataloader):
            x, y = x.to(device), y.to(device)
            decision = []

            for cropsize in cropsize_list:
              center_crop = torchvision.transforms.CenterCrop(cropsize)
              crop_image = center_crop(x)
              outputs = network(crop_image)
              decision.append(outputs)
              del outputs
            
            decision = sum(decision) / len(cropsize_list)
            num_correct += (torch.argmax(decision, axis=1) == y).sum().item()
            del x
            del y
      print('val loss', avg_loss)
      print('Epoch: {}, Validation Accuracy: {:.2f}'.format(epoch, num_correct / len(val_dataset)))
      acc = num_correct / len(val_dataset)
      scheduler.step(acc)
      if acc >= standard:
        torch.save(network,'resnet50_last_1')
        print('saving...', acc)
        standard = acc 


        

# tuning hyperparameters


## resnet 34 best


In [ ]:
# yields 0.82 of accuracy
# set up model
num_classes = len(train_dataset.classes)
feat_dim = 1200

#model
network_34 = ResNet34(num_classes, feat_dim=1200)
network_34 = network_34.to(device)

#hyperparameter
numEpochs = 25
in_features = 3 #RGB channels
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(network_34.parameters(), lr=0.01, weight_decay=5e-5, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
training(numEpochs, network_34)

## resnet 50 best

In [ ]:
# yields 0.83 of accuracy
# set up model
num_classes = len(train_dataset.classes)
feat_dim = 1200

#model
network_50 = ResNet50(num_classes, feat_dim=1200)
network_50 = network_50.to(device)

#hyperparameter
numEpochs = 25
in_features = 3 #RGB channels
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(network_50.parameters(), lr=0.01, weight_decay=5e-5, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
training(numEpochs, network_50)

## resnet 152 best

In [ ]:
# set up model
num_classes = len(train_dataset.classes)
feat_dim = 1200

#model
network_152 = ResNet152(num_classes, feat_dim=1200)
network_152 = network_152.to(device)

#hyperparameter
numEpochs = 25
in_features = 3 #RGB channels
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(network_152.parameters(), lr=0.01, weight_decay=5e-5, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
training(numEpochs, network_152)

## majority vote

In [ ]:
# Validate
avg_loss = 0
num_correct = 0
network_34.eval()
network_50.eval()
network_152.eval()
with torch.no_grad():
  cropsize_list = [24, 36, 48, 54, 64]

  for batch_num, (x, y) in enumerate(val_dataloader):
      x, y = x.to(device), y.to(device)
      decision_34 = []
      decision_50 = []
      decision_152 = []

      for cropsize in cropsize_list:
        center_crop = torchvision.transforms.CenterCrop(cropsize)
        crop_image = center_crop(x)
        outputs_34 = network_34(crop_image)
        outputs_50 = network_50(crop_image)
        outputs_152 = network_152(crop_image)
        decision_34.append(outputs_34)
        decision_50.append(outputs_50)
        decision_152.append(outputs_152)

        del outputs_34
        del outputs_50
        del outputs_152
      
      decision_34 = sum(decision_34) / len(cropsize_list)
      class_34 = torch.argmax(decision_34, axis = 1)
      decision_50 = sum(decision_50) / len(cropsize_list)
      class_50 = torch.argmax(decision_50, axis = 1)
      decision_152 = sum(decision_152) / len(cropsize_list)
      class_152 = torch.argmax(decision_152, axis = 1)

      #find majority vote
      final_class = []
      for i in range(len(class_34)):
        all_results = [int(class_34[i]),int(class_50[i]),int(class_152[i])]
        dict1 = dict(Counter(all_results))
        sorted_keys = sorted(dict1, key=dict1.get, reverse=True)
        final_class.append(sorted_keys[0])
        

      final_class = torch.tensor(final_class)
      final_class = final_class.to(device)
      num_correct += (final_class == y).sum().item()

      del x
      del y
      del decision_34
      del decision_50
      del decision_152

print('val loss', avg_loss)
print('Epoch: {}, Validation Accuracy: {:.2f}'.format(0, num_correct / len(val_dataset)))

val loss 0
Epoch: 0, Validation Accuracy: 0.85


# testing

In [ ]:
#read the test image

from torch.utils.data import DataLoader,Dataset
from PIL import Image
from os import listdir
from os.path import isfile, join
normalize = torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
class ImageDataset(Dataset):
    def __init__(self, file_list):
        self.file_list = file_list
        self.imgfiles = [f for f in listdir(self.file_list) if isfile(join(self.file_list, f))]
        

    def __len__(self):
        return len(self.imgfiles)

    def __getitem__(self, index):
        fname = self.file_list + '/'+ self.imgfiles[index]
        
        img = Image.open(fname)
        img = torchvision.transforms.ToTensor()(img)
        img = normalize(img)
      

        return img,self.imgfiles[index]

In [ ]:
testset = ImageDataset('test_data')
test_loader = torch.utils.data.DataLoader(testset, batch_size=64, 
                                               shuffle=False, num_workers=1)

In [ ]:
# Validate
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
avg_loss = 0
num_correct = 0
network_34.eval()
network_50.eval()
network_152.eval()
preds_submit = []
names = []
with torch.no_grad():
  cropsize_list = [24, 36, 48, 54, 64]

  for batch_num, (x, name) in enumerate(test_loader):
      x = x.to(device)
      decision_34 = []
      decision_50 = []
      decision_152 = []
      names.append(name)

      for cropsize in cropsize_list:
        center_crop = torchvision.transforms.CenterCrop(cropsize)
        crop_image = center_crop(x)
        outputs_34 = network_34(crop_image)
        outputs_50 = network_50(crop_image)
        outputs_152 = network_152(crop_image)
        decision_34.append(outputs_34)
        decision_50.append(outputs_50)
        decision_152.append(outputs_152)

        del outputs_34
        del outputs_50
        del outputs_152
      
      decision_34 = sum(decision_34) / len(cropsize_list)
      class_34 = torch.argmax(decision_34, axis = 1)
      decision_50 = sum(decision_50) / len(cropsize_list)
      class_50 = torch.argmax(decision_50, axis = 1)
      decision_152 = sum(decision_152) / len(cropsize_list)
      class_152 = torch.argmax(decision_152, axis = 1)

      #find majority vote
      final_class = []
      for i in range(len(class_34)):
        all_results = [int(class_34[i]),int(class_50[i]),int(class_152[i])]
        dict1 = dict(Counter(all_results))
        sorted_keys = sorted(dict1, key=dict1.get, reverse=True)

        
        final_class.append(sorted_keys[0])
      preds_submit.append(final_class)

      del x
      del name
      del decision_34
      del decision_50
      del decision_152


In [ ]:
network.eval()
preds_total = []
names = []
with torch.no_grad():
  for X_test, name in test_loader:
   # print(name)
    names.append(name)
    X_test = X_test.to(device='cuda')
    y_hat = network(X_test)
    
    y_hat = y_hat.cpu()
    y_hat = y_hat.detach().numpy()
    
    preds = np.argmax(y_hat, axis = 1)
   
    preds = preds.reshape(-1,1)
    preds_total.append(preds)

In [ ]:
dictclass = train_dataset.class_to_idx
dictclass.keys()
dictclass.values()

In [ ]:
finalnames = []
for k in names:
  #print(k)

  for n in range(0,len(k)):
    #print(k[n])

    finalnames.append(k[n])

In [ ]:

preds_final = []
for i in preds_submit:
  for t in i:
    
    add = [key for key, val in dictclass.items() if val == t]
    #print(add[0], t)
    preds_final.append(int(add[0]))
len(preds_final)

8000

In [ ]:
import pandas as pd
df = pd.DataFrame({'id':finalnames, "label":preds_final})
df.to_csv(r"submission_redo.csv", index=False)

In [ ]:
del train_dataset

# task2 pre-processing


In [ ]:
import os
import pandas as pd
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
from PIL import Image
import numpy as np
import torch.nn as nn
import torch

In [ ]:
from PIL import Image
import PIL.ImageOps  
import pandas as pd
dir = 'verification_pairs_val.txt'
compare = pd.read_csv(dir, header = None, delimiter = "\t")
text = []
for c in compare[0]:
  i1, i2, l = c.split(" ")
  text.append([i1,i2,l])

  
class TrainDataset(Dataset):
    
    def __init__(self,imageFolderDataset,text, test = False):
        
        
        self.imageFolderDataset = imageFolderDataset    
        self.image = [[t[0],t[1]] for t in text]
        self.label = [t[2] for t in text]
        self.test = test
        self.normalize = torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])


    def __getitem__(self,index):

        dirname0 =  text[index][0]
        dirname1 =  text[index][1]
        label = text[index][2]
       
        img0 = Image.open(dirname0)
        img1 = Image.open(dirname1)

        img0 = torchvision.transforms.ToTensor()(img0)
        img1 = torchvision.transforms.ToTensor()(img1)

        img0 = self.normalize(img0)
        img1 = self.normalize(img1)

    

        if self.test == False:
          l = torch.from_numpy(np.array(label, dtype = np.float32))
          return img0, img1, l
        else:
          return img0, img1 ,0
    
    def __len__(self):
        return len(self.image)

In [ ]:


train_dataset = TrainDataset(dir, text)
train_dataloader = DataLoader(train_dataset,
                        shuffle=False,
                        num_workers=0,
                        batch_size=1 )

# task2 Cosine Similarity

In [ ]:
compute_sim = nn.CosineSimilarity(dim=0)
#print("CS b/n two images of class 0: {:.4f}".format(compute_sim(feats_0, feats_1)))
def testVerify(model_34,model_50,model_152, train_dataloader):
    similarity = np.array([])
    true = np.array([])
    cropsize_list = [24, 36, 48, 54, 64]
    
    model_34.eval()
    model_50.eval()
    model_152.eval()
    with torch.no_grad():
        for batch_idx, (img0, img1, targets) in enumerate(train_dataloader):
            img0, img1, targets = img0.cuda(), img1.cuda(), targets.cuda()
            

            #decisions_0 = []
            #decisions_1 = []
            decisions_34_0 = []
            decisions_50_0 = []
            decisions_152_0 = []
            decisions_34_1 = []
            decisions_50_1 = []
            decisions_152_1 = []

            for cropsize in cropsize_list:
              center_crop = torchvision.transforms.CenterCrop(cropsize)
              crop_img0 = center_crop(img0)
              crop_img1 = center_crop(img1)

              # resnet34
              feats_34_0 = model_34(crop_img0, return_embedding=False).squeeze(0)
              feats_34_1 = model_34(crop_img1, return_embedding=False).squeeze(0)
              decisions_34_0.append(feats_34_0)
              decisions_34_1.append(feats_34_1)

              # resnet50
              feats_50_0 = model_50(crop_img0, return_embedding=False).squeeze(0)
              feats_50_1 = model_50(crop_img1, return_embedding=False).squeeze(0)
              decisions_50_0.append(feats_50_0)
              decisions_50_1.append(feats_50_1)

              # resnet152
              feats_152_0 = model_152(crop_img0, return_embedding=False).squeeze(0)
              feats_152_1 = model_152(crop_img1, return_embedding=False).squeeze(0)
              decisions_152_0.append(feats_152_0)
              decisions_152_1.append(feats_152_1)



              del crop_img0
              del crop_img1
              del feats_34_0
              del feats_34_1
              del feats_50_0
              del feats_50_1
              del feats_152_0
              del feats_152_1

            decisions_34_0 = sum(decisions_34_0) / len(cropsize_list)
            decisions_34_1 = sum(decisions_34_1) / len(cropsize_list)
            decisions_50_0 = sum(decisions_50_0) / len(cropsize_list)
            decisions_50_1 = sum(decisions_50_1) / len(cropsize_list)
            decisions_152_0 = sum(decisions_152_0) / len(cropsize_list)
            decisions_152_1 = sum(decisions_152_1) / len(cropsize_list)

            decision_0 = sum([decisions_34_0,decisions_50_0,decisions_152_0])/3
            decision_1 = sum([decisions_34_1,decisions_50_1,decisions_152_1])/3

      
            sim = compute_sim(decision_0,decision_1)
                     
            similarity = np.concatenate((similarity, sim.cpu().numpy().reshape(-1)))

            true = np.concatenate((true, targets.cpu().numpy().reshape(-1)))
            
            del img0
            del img1
            del targets
     
    return similarity, true

In [ ]:
compute_sim = nn.CosineSimilarity(dim=0)
#print("CS b/n two images of class 0: {:.4f}".format(compute_sim(feats_0, feats_1)))
def testVerify(model, train_dataloader):
    similarity = np.array([])
    true = np.array([])
    
    model.eval()
    with torch.no_grad():
        for batch_idx, (img0, img1, targets) in enumerate(train_dataloader):
            img0, img1, targets = img0.cuda(), img1.cuda(), targets.cuda()
          
          
            feats_0 = model(img0, return_embedding=False).squeeze(0)
            feats_1 = model(img1, return_embedding=False).squeeze(0)
      
            sim = compute_sim(feats_0,feats_1)
                     
            similarity = np.concatenate((similarity, sim.cpu().numpy().reshape(-1)))

            true = np.concatenate((true, targets.cpu().numpy().reshape(-1)))
            
            del img0
            del img1
            del targets
    return similarity, true

In [ ]:
compute_sim = nn.CosineSimilarity(dim=0)
#print("CS b/n two images of class 0: {:.4f}".format(compute_sim(feats_0, feats_1)))
def testVerify(model_50, train_dataloader):
    similarity = np.array([])
    true = np.array([])
    cropsize_list = [24, 36, 48, 54, 64]

    model_50.eval()

    with torch.no_grad():
        for batch_idx, (img0, img1, targets) in enumerate(train_dataloader):
            img0, img1, targets = img0.cuda(), img1.cuda(), targets.cuda()
       
            decisions_50_0 = []
            decisions_50_1 = []

            for cropsize in cropsize_list:
              center_crop = torchvision.transforms.CenterCrop(cropsize)
              crop_img0 = center_crop(img0)
              crop_img1 = center_crop(img1)

              # resnet50
              feats_50_0 = model_50(crop_img0, return_embedding=False).squeeze(0)
              feats_50_1 = model_50(crop_img1, return_embedding=False).squeeze(0)
              decisions_50_0.append(feats_50_0)
              decisions_50_1.append(feats_50_1)

              del crop_img0
              del crop_img1
              del feats_50_0
              del feats_50_1

            decisions_50_0 = sum(decisions_50_0) / len(cropsize_list)
            decisions_50_1 = sum(decisions_50_1) / len(cropsize_list)
      
            sim = compute_sim(decisions_50_0,decisions_50_1)
                     
            similarity = np.concatenate((similarity, sim.cpu().numpy().reshape(-1)))

            true = np.concatenate((true, targets.cpu().numpy().reshape(-1)))
            
            del img0
            del img1
            del targets
     
    return similarity, true

In [ ]:
#%cd .. 
#%cd 11785-spring2021-hw2p2s2-face-verification
similarity, true = testVerify(network_50, train_dataloader)

In [ ]:
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(true, similarity)
print(auc)


0.931781866230351


# task2 test

In [ ]:
%cd .. 
%cd 11785-spring2021-hw2p2s2-face-verification
#network = torch.load('resnet_0.7x2')

/content/gdrive/My Drive/competitions
/content/gdrive/My Drive/competitions/11785-spring2021-hw2p2s2-face-verification


In [ ]:
dir = 'verification_pairs_test.txt'
text = []
compare = pd.read_csv(dir, header = None, delimiter = "\t")
for c in compare[0]:
  i1, i2 = c.split(" ")
  text.append([i1,i2,0])

test_dataset = TrainDataset(dir, text, test = True)
test_dataloader = DataLoader(test_dataset,
                        shuffle=False,
                        num_workers=0,
                        batch_size=1)

In [ ]:

def testVerify_final(model, train_dataloader):
    similarity = np.array([])
    true = np.array([])
    
    model.eval()
    with torch.no_grad():
        for batch_idx, (img0, img1, targets) in enumerate(train_dataloader):
            img0, img1, targets = img0.cuda(), img1.cuda(), targets.cuda()
            # find cos similarity between embeddings
            #print(img0.shape)

          
            feats_0 = model(img0, return_embedding=True)[0].squeeze(0)
            feats_1 = model(img1, return_embedding=True)[0].squeeze(0)
      
            sim = compute_sim(feats_0,feats_1)
           # print('sim', sim)

            similarity = np.concatenate((similarity, sim.cpu().numpy().reshape(-1)))
    
            
            del img0
            del img1
            del targets
    return similarity

In [ ]:
compute_sim = nn.CosineSimilarity(dim=0)
#print("CS b/n two images of class 0: {:.4f}".format(compute_sim(feats_0, feats_1)))
def testVerify_final(model_34,model_50,model_152, train_dataloader):
    similarity = np.array([])
    true = np.array([])
    cropsize_list = [24, 36, 48, 54, 64]
    
    model_34.eval()
    model_50.eval()
    model_152.eval()
    with torch.no_grad():
        for batch_idx, (img0, img1, targets) in enumerate(train_dataloader):
            print(batch_idx)
            img0, img1, targets = img0.cuda(), img1.cuda(), targets.cuda()
            
            decisions_34_0 = []
            decisions_50_0 = []
            decisions_152_0 = []
            decisions_34_1 = []
            decisions_50_1 = []
            decisions_152_1 = []

            for cropsize in cropsize_list:
              center_crop = torchvision.transforms.CenterCrop(cropsize)
              crop_img0 = center_crop(img0)
              crop_img1 = center_crop(img1)

              # resnet34
              feats_34_0 = model_34(crop_img0, return_embedding=False).squeeze(0)
              feats_34_1 = model_34(crop_img1, return_embedding=False).squeeze(0)
              decisions_34_0.append(feats_34_0)
              decisions_34_1.append(feats_34_1)

              # resnet50
              feats_50_0 = model_50(crop_img0, return_embedding=False).squeeze(0)
              feats_50_1 = model_50(crop_img1, return_embedding=False).squeeze(0)
              decisions_50_0.append(feats_50_0)
              decisions_50_1.append(feats_50_1)

              # resnet152
              feats_152_0 = model_152(crop_img0, return_embedding=False).squeeze(0)
              feats_152_1 = model_152(crop_img1, return_embedding=False).squeeze(0)
              decisions_152_0.append(feats_152_0)
              decisions_152_1.append(feats_152_1)



              del crop_img0
              del crop_img1
              del feats_34_0
              del feats_34_1
              del feats_50_0
              del feats_50_1
              del feats_152_0
              del feats_152_1

            decisions_34_0 = sum(decisions_34_0) / len(cropsize_list)
            decisions_34_1 = sum(decisions_34_1) / len(cropsize_list)
            decisions_50_0 = sum(decisions_50_0) / len(cropsize_list)
            decisions_50_1 = sum(decisions_50_1) / len(cropsize_list)
            decisions_152_0 = sum(decisions_152_0) / len(cropsize_list)
            decisions_152_1 = sum(decisions_152_1) / len(cropsize_list)

            decision_0 = sum([decisions_34_0,decisions_50_0,decisions_152_0])/3
            decision_1 = sum([decisions_34_1,decisions_50_1,decisions_152_1])/3

      
            sim = compute_sim(decision_0,decision_1)
                     
            similarity = np.concatenate((similarity, sim.cpu().numpy().reshape(-1)))

            
            del img0
            del img1
            del targets
     
    return similarity

In [ ]:
similarity = testVerify_final(network_34,network_50,network_152, test_dataloader)

In [ ]:
import pandas as pd
df = pd.DataFrame({'Id':compare[0], "Category":similarity})
df.to_csv(r"submission4.csv", index=False)

In [ ]:
!kaggle competitions submit -c hw2p2-summer2021 -f submission4.csv -m "Message"

100% 4.03M/4.03M [00:06<00:00, 653kB/s]
Successfully submitted to HW2p2_summer2021